In [ ]:
import math 
class Stock:
	Total_Investment = 0
	Current_Investment = 0
	Total_Liquidity = 0
	profit_loss = 0 
	def __init__(this,Stock_Name,Quantity,Buy_Value,Sell_Value,Total_Sell,Total_Buy):
		this.Stock_Name = Stock_Name 
		this.Quantity = int(Quantity)
		this.Buy_Value = float(Buy_Value)
		this.Sell_Value = float(Sell_Value)
		this.Total_Sell = float(Total_Sell)
		this.Total_Buy = float(Total_Buy)
		Stock.Total_Investment += float(Total_Buy)
		Stock.Current_Investment += float(Total_Buy)
	def Sell_Stock( this , q , price ):
		if( q > this.Quantity ):
			return False 
		else:
			this.Quantity -= q 
			Stock.Current_Investment -= ( q * this.Buy_Value )
			this.Sell_Value = price
			this.Total_Sell += ( q * this.Sell_Value )
			Stock.Total_Liquidity += this.Total_Sell
			Stock.profit_loss += ( ( q * this.Sell_Value ) - ( q * this.Buy_Value ) ) 
			return True 
	def Buy_Stock( this , q , price ):
		this.Quantity += q 
		Stock.Current_Investment += float( q * price )
		Stock.Total_Investment += float( q * price )
		this.Buy_Value = ( this.Total_Buy + float( q * price ) )/float( this.Quantity )
		this.Total_Buy += ( q * price )
		print('\n Bought Successfully\n')
		return True
	def Display_Name(this):
		print(this.Stock_Name , '\n')
	def Display(this):
		print(this.Stock_Name , ' '*( 14 - len( this.Stock_Name ) ) , math.trunc(this.Quantity) , ' '*( 13 ) , round(this.Buy_Value,3) , ' '*( 16 ) ,round(this.Total_Buy,3) , '\n' )
	@staticmethod
	def Investment_Insight( ):
		print( round( Stock.Total_Investment,3 ) , ' '*( 21 ) , round(Stock.Total_Liquidity,3) , ' '*( 22 ) , round(Stock.Current_Investment,3) , ' '*( 20 ) , Stock.profit_loss , '\t\n' )
	def Weightage(this):
		print(this.Stock_Name , ' '*( 21 - len(this.Stock_Name) ) , this.Perecentage( ) , '\n')
	def Perecentage(this):
		return ( ( ( float(this.Quantity) * float(this.Buy_Value) ) / Stock.Current_Investment )  * int(100) )
	def Get_Quantity(this):
		return this.Quantity
def main( ):
	v = [];
	print('\t\t\t--------------------WELCOME TO INVESTMENT BOOK--------------------\n')
	while(True):
		print("\nENTER YOUR CHOICE :\n1.See Portfolio\n2.Show Weightage\n3.Investment insight\n4.Buy Stock\n5.Sell Stock\n6.Exit\n")
		choice = int( input( ) ) 
		v.sort( reverse = True )
		if choice == 1 :
			print( "\nStock_Name\tQuantity\tBuy_Value\tCurrent_Total\n" )
			for x in v :
				x.Display( )
		elif( choice == 2 ):
			if( len( v ) == 0 ):
				print('NO STOCK AVAILABLE\n')
			else:
				for x in v :
					x.Weightage( )
		elif( choice == 3 ):
			print( "\nTotal_Investment\tTotal_Liquidity\t\tCurrent_Investment\tprofit_loss\n" )
			Stock.Investment_Insight()
		elif( choice == 4 ): 
			m = input("\nWANT TO BUY NEW STOCK PRESS 1 , FOR EXSISTING STOCK PRESS 2 \n" )
			if int(m) == 1:
				if len( v ) == 0 :
					print( "\nNO STOCK IN PORTFOLIO \n" )
				else:
					print( "\nWHICH STOCK YOU WANT TO BUY : \n" )
					for x in range( len( v ) ) :
						print( x + 1 , end = " ")
						v[ x ].Display_Name( )
					t = int( input( ) ) 
					if( t > len( v ) ):
						print( "\nWRONG INPUT\n")
					else:
						q = float( input( "\nENTER Quantity\n" ) ) 
						price = float( input( "\nENTER BUY PRICE\n" ) )
						v[ t - 1 ].Buy_Stock( q , price )
			else:
				s = input("\nENTER STOCK NAME:\n" ) 
				q = input( "\nENTER Quantity\n" ) 
				price = input( ("\nENTER BUY PRICE\n" ) )
				value = float( q ) * float( price ) 
				print(value)
				obj = Stock( s , q , price , 0 , 0  , value )
				v.append( obj )
		elif choice == 5:
			if( len( v ) == 0 ):
				print( "\nNO STOCK IN PORTFOLIO\n")
			else:
				print("\nWHICH STOCK YOU WANT TO SELL\n")
				for i in range( len( v ) ):
					print( i + 1 , " " )
					v[ i ].Display_Name( ) 
				t = int( input( ) )
				if t > len( v ):
					print('\t\t\t--------------------WRONG INPUT-------------------')
				q = int( input('ENTER QUANTITY') )
				p = float( input('ENTER PRICE') )
				if v[ t - 1 ].Sell_Stock( q , p ):
					print( 'SOLD SUCCESSFULLY' )
				else:
					print( '\t\t\t--------------------INVALID INPUT-----------------')
				if v[ t - 1 ].Get_Quantity( ) == 0 :
					del v[ t - 1 ] 
		elif choice == 6 :
			break 
		else:
			print("galat chal rha hai bhsdk")
if __name__ == "__main__":
    main()
